In [24]:
### we start with the liar dataset

import numpy as np
import pandas as pd

In [25]:
#get the dataset
df_train = pd.read_csv('../../../datasets/liar_tweaked/trainvectordata.csv')
df_test = pd.read_csv('../../../datasets/liar_tweaked/testvectordata.csv')
df_valid = pd.read_csv('../../../datasets/liar_tweaked/validvectordata.csv')

In [26]:
X_train=df_train['statement']
X_test=df_test['statement']
Y_train=df_train['label']
Y_test=df_test['label']
X_valid=df_valid['statement']
Y_valid=df_valid['label']

In [27]:
X_test=X_test.append(X_valid, ignore_index=True)

In [28]:
Y_test=Y_test.append(Y_valid, ignore_index=True)

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical



In [30]:
X_test = X_test.str.replace('[^a-zA-Z]',' ').str.lower()
X_train = X_train.str.replace('[^a-zA-Z]',' ').str.lower()

In [31]:
import nltk
import re
from nltk.corpus import stopwords

In [32]:
# Remove stop words with regex. '\\b' matches any break (space or linebreak or whatever) and '|'
# is an or operator. So, for example '\\ba\\b|\\bis\\b|\\band\\b' will match 'a', 'is' or 'and'.
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
X_test = X_test.str.replace(stop_re, '')
X_train = X_train.str.replace(stop_re, '')
X_test.head()

0    building  wall   u   mexico border  take liter...
1     wisconsin   pace  double  number  layoffs  year 
2          says john mccain  done nothing  help  vets 
3    suzanne bonamici supports  plan   cut choice  ...
4     asked   reporter whether hes   center   crimi...
Name: statement, dtype: object

In [33]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
for i in range(0, len(X_test)):
    s=X_test[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X_test[i]=st
for i in range(0, len(X_train)):
    s=X_train[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X_train[i]=st

In [34]:
# Tokenize words
X_test = X_test.str.split()
X_train = X_train.str.split()

In [35]:
import gensim
from gensim.models import Word2Vec

In [36]:
w2vtest = gensim.models.Word2Vec(X_test.tolist(), min_count=1)
w2vtrain = gensim.models.Word2Vec(X_train.tolist(), min_count=1)

In [37]:
def transform_data(data,w2v):
    for i in range(0, len(data)):
        x=data[i]
        b = np.zeros(100) 
        for w in x:
            b+=w2v[w]
        b = np.divide(b, 100)
        x=b
        data[i]=x
        #print(b)
    return data

In [38]:
X_test=transform_data(X_test,w2vtest)
X_train=transform_data(X_train,w2vtrain)

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [39]:
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

In [40]:
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [41]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
## Creating model

# ## expand train dimnesion: pass from 2d to 3d
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
model=Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=False), input_shape=(1, 100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 161,001
Trainable params: 161,001
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=20,batch_size=64)

Train on 10229 samples, validate on 2551 samples
Epoch 1/20
10229/10229 [==============================] - 12s 1ms/sample - loss: 0.6852 - accuracy: 0.5603 - val_loss: 0.6898 - val_accuracy: 0.5417
Epoch 2/20
10229/10229 [==============================] - 3s 308us/sample - loss: 0.6832 - accuracy: 0.5619 - val_loss: 0.6903 - val_accuracy: 0.5417
Epoch 3/20
10229/10229 [==============================] - 3s 302us/sample - loss: 0.6819 - accuracy: 0.5622 - val_loss: 0.6905 - val_accuracy: 0.5417
Epoch 4/20
10229/10229 [==============================] - 3s 332us/sample - loss: 0.6807 - accuracy: 0.5658 - val_loss: 0.6909 - val_accuracy: 0.5417
Epoch 5/20
10229/10229 [==============================] - 3s 324us/sample - loss: 0.6785 - accuracy: 0.5676 - val_loss: 0.6921 - val_accuracy: 0.5414
Epoch 6/20
10229/10229 [==============================] - 3s 320us/sample - loss: 0.6762 - accuracy: 0.5727 - val_loss: 0.6932 - val_accuracy: 0.5382
Epoch 7/20
10229/10229 [============================

In [43]:
### now we have the kaggle dataset

In [44]:
df = pd.read_csv('../../../datasets/kaggle/train.csv')



In [45]:
df['content'] = df['title'] + ' ' + df['text']
df=df[df['content']==df['content']]
df=df[df['label']==df['label']]

In [46]:
df.to_csv('../../../datasets/kaggle/final.csv')

In [47]:
df=pd.read_csv('../../../datasets/kaggle/final.csv')

In [48]:
X=df['content']
Y=df['label']

In [49]:
X = X.str.replace('[^a-zA-Z]',' ').str.lower()



In [50]:
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
X = X.str.replace(stop_re, '')
X.head()

0    house dem aide     even see comey  letter  jas...
1    flynn  hillary clinton  big woman  campus   br...
2      truth might get  fired   truth might get  fi...
3       civilians killed  single us airstrike   ide...
4    iranian woman jailed  fictional unpublished st...
Name: content, dtype: object

In [51]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
for i in range(0, len(X)):
    s=X[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X[i]=st

In [52]:
# Tokenize words
X = X.str.split()

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [55]:
w2vtest = gensim.models.Word2Vec(X_test.tolist(), min_count=1)
w2vtrain = gensim.models.Word2Vec(X_train.tolist(), min_count=1)

In [56]:
X_test=X_test.reset_index()
Y_test=Y_test.reset_index()
X_train=X_train.reset_index()
Y_train=Y_train.reset_index()

In [57]:
Y_test=Y_test['label']
X_test=X_test['content']
X_train=X_train['content']
Y_train=Y_train['label']

In [58]:
X_test=transform_data(X_test,w2vtest)
X_train=transform_data(X_train,w2vtrain)

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [59]:
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
## Creating model

# ## expand train dimnesion: pass from 2d to 3d
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
model=Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=False), input_shape=(1, 100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 161,001
Trainable params: 161,001
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=20,batch_size=64)

Train on 15152 samples, validate on 5051 samples
Epoch 1/20
15152/15152 [==============================] - 13s 862us/sample - loss: 0.3388 - accuracy: 0.8652 - val_loss: 0.5754 - val_accuracy: 0.7387
Epoch 2/20
15152/15152 [==============================] - 5s 306us/sample - loss: 0.2527 - accuracy: 0.9025 - val_loss: 0.7448 - val_accuracy: 0.7123
Epoch 3/20
15152/15152 [==============================] - 5s 350us/sample - loss: 0.2285 - accuracy: 0.9073 - val_loss: 0.8407 - val_accuracy: 0.6989
Epoch 4/20
15152/15152 [==============================] - 5s 335us/sample - loss: 0.2135 - accuracy: 0.9152 - val_loss: 0.9521 - val_accuracy: 0.6892
Epoch 5/20
15152/15152 [==============================] - 6s 364us/sample - loss: 0.2043 - accuracy: 0.9171 - val_loss: 0.7854 - val_accuracy: 0.7090
Epoch 6/20
15152/15152 [==============================] - 5s 308us/sample - loss: 0.1952 - accuracy: 0.9211 - val_loss: 0.9049 - val_accuracy: 0.6923
Epoch 7/20
15152/15152 [==========================